# Import libraries

In [1]:
import numpy as np
import pandas as pd
from glob import glob
from nltk.metrics import edit_distance

_______

# Import dataset

The dataset is in 2 different excel files.

In [2]:
stock = sorted(glob("marketing_campaign_*.xlsx"))
stock

['marketing_campaign_1.xlsx', 'marketing_campaign_2.xlsx']

In [3]:
market = pd.concat((pd.read_excel(file).assign(dilename = file)
         for file in stock),
          ignore_index = True)

In [4]:
market.shape

(2240, 30)

We want to increase the number of information we can see from tables.

We maximize the number of rows and columns xe can see when we print the tables.

In [5]:
pd.set_option("display.max_row", market.shape[0]+1)
pd.set_option("display.max_column", market.shape[1]+1)

In [6]:
market.head()

,ID,Year Birth,Education,Marital Status,Income,Kid at home,Teen at home,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,dilename
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1,marketing_campaign_1.xlsx
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0,marketing_campaign_1.xlsx
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0,marketing_campaign_1.xlsx
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,3,11,0,marketing_campaign_1.xlsx
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,3,11,0,marketing_campaign_1.xlsx


_______

# Change names

Some names 2 or 3 parts names have a space betweeen the parts.

We want solve that by changing these names.

In [7]:
market.rename({"Year Birth" : "Year_Birth",
               "Marital Status" : "Marital_Status",
               "Kid at home" : "Kidhome",
               "Teen at home" : "Teenhome"},
             axis = 1,
             inplace = True)

In [8]:
market.head(0)

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,dilename


_______

# The features

In [9]:
market.columns

Index(['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response',
       'dilename'],
      dtype='object')

# THE DATA

## Amount spent on various products

MntFishProducts - amount spent on fish products in the last 2 years

MntMeatProducts - amount spent on meat products in the last 2 years

MntFruits - amount spent on fruits products in the last 2 years

MntSweetProducts - amount spent on sweet products in the last 2 years

MntWines - amount spent on wine products in the last 2 years

MntGoldProds - amount spent on gold products in the last 2 years

## Number of purchases from various sources

NumDealsPurchases - number of purchases made with discount

NumCatalogPurchases - number of purchases made using catalogue

NumStorePurchases - number of purchases made directly in stores

NumWebPurchases - number of purchases made through company’s web site

NumWebVisitsMonth - number of visits to company’s web site in the last month

## Accept campaign

AcceptedCmp1 - 1 if customer accepted the offer in the 1st campaign, 0 otherwise

AcceptedCmp2 - 1 if customer accepted the offer in the 2nd campaign, 0 otherwise

AcceptedCmp3 - 1 if customer accepted the offer in the 3rd campaign, 0 otherwise

AcceptedCmp4 - 1 if customer accepted the offer in the 4th campaign, 0 otherwise

AcceptedCmp5 - 1 if customer accepted the offer in the 5th campaign, 0 otherwise

## Other informations

Response (target) - 1 if customer accepted the offer in the last campaign, 0 otherwise

Complain - 1 if customer complained in the last 2 years

DtCustomer - date of customer’s enrolment with the company

Education - customer’s level of education

Marital - customer’s marital status

Kidhome - number of small children in customer’s household

Teenhome - number of teenagers in customer’s household

Income - customer’s yearly household income

Recency - number of days since the last purchase

_______

# Drop useless columns

We got some useless columns.

They dont give any information because they repeat the same value over the column.

(The value 3 for Z_CostContact, and 11 for Z_Revenue).

We have to drop them.

In [10]:
market.Z_CostContact.value_counts()

3    2240
Name: Z_CostContact, dtype: int64

In [11]:
market.Z_Revenue.value_counts()

11    2240
Name: Z_Revenue, dtype: int64

In [12]:
market.drop(["Z_CostContact","Z_Revenue"],
           axis = 1,
           inplace = True)

In [13]:
# Now, we got 28 features (versus 30 before the drop)
market.shape

(2240, 28)

In [14]:
market.head(0)

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,dilename


_______

# Missing values

In [15]:
market.isnull().sum()

ID                      0
Year_Birth              0
Education               0
Marital_Status          0
Income                 24
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Complain                0
Response                0
dilename                0
dtype: int64

Only the Income got missing values (24)

We have decide to drop the lines where the Income values are missing.

In [16]:
market.dropna(axis=0,
             inplace = True)

In [17]:
market.Income.isnull().sum()

0

In [18]:
market.shape

(2216, 28)

_______

# We have to see the features individually

In [19]:
market.dtypes

ID                       int64
Year_Birth               int64
Education               object
Marital_Status          object
Income                 float64
Kidhome                  int64
Teenhome                 int64
Dt_Customer             object
Recency                  int64
MntWines                 int64
MntFruits                int64
MntMeatProducts          int64
MntFishProducts          int64
MntSweetProducts         int64
MntGoldProds             int64
NumDealsPurchases        int64
NumWebPurchases          int64
NumCatalogPurchases      int64
NumStorePurchases        int64
NumWebVisitsMonth        int64
AcceptedCmp3             int64
AcceptedCmp4             int64
AcceptedCmp5             int64
AcceptedCmp1             int64
AcceptedCmp2             int64
Complain                 int64
Response                 int64
dilename                object
dtype: object

# Categorial features

We begin by looking at the categories of the categorial features.

We got 4 categorial features :
> Kidhome

> Teenhome

> Education

> Marital_Status

In [20]:
market.Kidhome.value_counts()

0      1282
1       887
2        46
222       1
Name: Kidhome, dtype: int64

In [21]:
market.Teenhome.value_counts()

0     1147
1     1017
2       51
97       1
Name: Teenhome, dtype: int64

In [22]:
market.Marital_Status.value_counts()

Married     856
Together    572
Single      471
Divorced    232
Widow        76
Alone         3
YOLO          2
Absurd        2
Togther       1
Married       1
Name: Marital_Status, dtype: int64

In [23]:
market.Education.value_counts()

Graduation    1116
PhD            480
Master         364
2n Cycle       200
Basic           54
Master           1
PfD              1
Name: Education, dtype: int64

These features give us some various problem.

Kidhome and Teenhome seems to have outliers, with an exterme value inside. It seems impossible to have 222 kids or 97 Teens at home.

In the Marital status values, Married appear 2 times, same orthographe but 2 different values. And it seeme than "Togther" is a typo mistake, we assume than the correct answer was "Together".

In Education we see the same problems than in marital status, with Master which appear 2 times and "PfD" instead of "PhD"

______

# Here, we apply corrections to clean the dataset :

## CHILDS (Kidhome and Teenhome problem)

We want to drop the lines where the family have 222 Kids at home and the one where there is 97 Teens at home.

First, we need to found these values inside the dataset :

In [24]:
market[market["Kidhome"] == market["Kidhome"].max()]

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,dilename
77,7503,1976,Graduation,Single,75825.0,222,0,2012-10-12,40,1032,105,779,137,105,51,0,5,8,9,4,1,0,0,0,0,0,1,marketing_campaign_1.xlsx


In [25]:
market[market["Teenhome"] == market["Teenhome"].max()]

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,dilename
38,8595,1973,Graduation,Widow,42429.0,0,97,2014-02-11,99,55,0,6,2,0,4,2,1,1,3,5,0,0,0,0,0,0,0,marketing_campaign_1.xlsx


In [26]:
# We have to drop the lines number 77 and 38 :
market.drop([38, 77], axis = 0, inplace = True)

In [27]:
market.shape

(2214, 28)

_______

## EDUCATION

It seems than we have problems with some value inside the Education's feature :

> 'Master' appears 2 times

> 'PfD' seems to be an error 

### The "Master " problem
We find than the 2nd Master have a whitspace inside :

In [28]:
# The 2nd "Master" have a space : "Master "
market[market.Education == "Master "]

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,dilename
51,4114,1964,Master,Married,79143.0,0,0,2012-08-11,2,650,37,780,27,167,32,1,6,9,13,3,0,0,0,0,0,0,0,marketing_campaign_1.xlsx


In [29]:
#To drop the whitespace at the beginning and the end :
market["Education"] = market["Education"].str.strip()

In [30]:
market.Education.value_counts()

Graduation    1114
PhD            480
Master         365
2n Cycle       200
Basic           54
PfD              1
Name: Education, dtype: int64

_______

### The "PfD" problem

We have to transform the value "semantically near" **PhD** into a **PhD** value.

In [31]:
EducX = pd.DataFrame(data = {"Education" : market.Education[:]})

In [32]:
EducX.head()

,Education
0,Graduation
1,Graduation
2,Graduation
3,Graduation
4,PhD


In [33]:
EducX.Education.value_counts()

Graduation    1114
PhD            480
Master         365
2n Cycle       200
Basic           54
PfD              1
Name: Education, dtype: int64

In [34]:
EducX["Education_distance_PhD"] = EducX["Education"].map(lambda x : edit_distance(x, "PhD"))

In [35]:
EducX.head()

,Education,Education_distance_PhD
0,Graduation,10
1,Graduation,10
2,Graduation,10
3,Graduation,10
4,PhD,0


Now, we have to test how many words have non-significative differences with PhD. They can be some typo mistakes.

In [36]:
EducX.Education_distance_PhD.value_counts()

10    1114
0      480
6      365
8      200
5       54
1        1
Name: Education_distance_PhD, dtype: int64

We have a value with just 1 letter of difference with PhD.

The other words have to many difference (Between 5 and 10), so they can't be a typo mistake.

We decide to change the words with less than 2 differences with "PhD". 

In [37]:
msk = EducX["Education_distance_PhD"] <= 1

In [38]:
msk.head()

0    False
1    False
2    False
3    False
4     True
Name: Education_distance_PhD, dtype: bool

In [39]:
EducX.loc[msk, "Education"] = "PhD"

In [40]:
EducX.head()

,Education,Education_distance_PhD
0,Graduation,10
1,Graduation,10
2,Graduation,10
3,Graduation,10
4,PhD,0


In [41]:
EducX.Education.value_counts()

Graduation    1114
PhD            481
Master         365
2n Cycle       200
Basic           54
Name: Education, dtype: int64

Now, we can replace the feature "Education" inside **market** by "Education" from **EducX**

In [42]:
EducX.columns

Index(['Education', 'Education_distance_PhD'], dtype='object')

In [43]:
EducY = EducX.drop("Education_distance_PhD",
                  axis = 1)
EducY.columns

Index(['Education'], dtype='object')

In [44]:
market["Education"] = EducY[:]
market.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,dilename
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,1,marketing_campaign_1.xlsx
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,0,marketing_campaign_1.xlsx
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,0,marketing_campaign_1.xlsx
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,0,marketing_campaign_1.xlsx
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,0,marketing_campaign_1.xlsx


In [45]:
market.Education.value_counts()

Graduation    1114
PhD            481
Master         365
2n Cycle       200
Basic           54
Name: Education, dtype: int64

_______

## MARITAL STATUS

In [46]:
market.Marital_Status.value_counts()

Married     856
Together    572
Single      470
Divorced    232
Widow        75
Alone         3
YOLO          2
Absurd        2
Togther       1
Married       1
Name: Marital_Status, dtype: int64

> We got 2 "Married"

> Togther seems to be a typo

### The Married problem :

In [47]:
# The 2nd "Married" have a space and become "Married "
market[market.Marital_Status == "Married "]

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,dilename
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,0,marketing_campaign_1.xlsx


In [48]:
#To drop the whitespace at the beginning and the end :
market["Marital_Status"] = market["Marital_Status"].str.strip()

In [49]:
market.Marital_Status.value_counts()

Married     857
Together    572
Single      470
Divorced    232
Widow        75
Alone         3
YOLO          2
Absurd        2
Togther       1
Name: Marital_Status, dtype: int64

_______

### The Together problem :

In [50]:
Couple = pd.DataFrame(data = {"Marital_Status" : market.Marital_Status[:]})

In [51]:
Couple.head()

,Marital_Status
0,Single
1,Single
2,Together
3,Together
4,Married


In [52]:
Couple.Marital_Status.value_counts()

Married     857
Together    572
Single      470
Divorced    232
Widow        75
Alone         3
YOLO          2
Absurd        2
Togther       1
Name: Marital_Status, dtype: int64

In [53]:
Couple["Marital_Status_distance_Together"] = Couple["Marital_Status"].map(lambda x : edit_distance(x, "Together"))

In [54]:
Couple.head()

,Marital_Status,Marital_Status_distance_Together
0,Single,7
1,Single,7
2,Together,0
3,Together,0
4,Married,7


In [55]:
Couple.Marital_Status_distance_Together.value_counts()

7    1562
0     572
8      79
1       1
Name: Marital_Status_distance_Together, dtype: int64

In [56]:
msk = Couple["Marital_Status_distance_Together"] <= 1

In [57]:
msk.head()

0    False
1    False
2     True
3     True
4    False
Name: Marital_Status_distance_Together, dtype: bool

In [58]:
Couple.loc[msk, "Marital_Status"] = "Together"

In [59]:
Couple.head()

,Marital_Status,Marital_Status_distance_Together
0,Single,7
1,Single,7
2,Together,0
3,Together,0
4,Married,7


In [60]:
Couple.Marital_Status.value_counts()

Married     857
Together    573
Single      470
Divorced    232
Widow        75
Alone         3
YOLO          2
Absurd        2
Name: Marital_Status, dtype: int64

Now, i can replace the feature "Marital_Status" inside **market** by "Marital_Status" from **Couple**

In [61]:
Couple_x = Couple.drop("Marital_Status_distance_Together",
                  axis = 1)
Couple_x.columns

Index(['Marital_Status'], dtype='object')

In [62]:
market["Marital_Status"] = Couple_x[:]
market.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,dilename
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,1,marketing_campaign_1.xlsx
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,0,marketing_campaign_1.xlsx
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,0,marketing_campaign_1.xlsx
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,0,marketing_campaign_1.xlsx
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,0,marketing_campaign_1.xlsx


In [63]:
market.Marital_Status.value_counts()

Married     857
Together    573
Single      470
Divorced    232
Widow        75
Alone         3
YOLO          2
Absurd        2
Name: Marital_Status, dtype: int64

_______

# To make a global analysis, we have to modify/create some variables. 

# Creation of the ChildsNumber column

In [64]:
market["ChildsNumber"] = market["Kidhome"] + market["Teenhome"]

**"ChildsNumber"** represent the total number of childs at home. Kids + Teens.

_______

# Creation of the MntTotal column

In [65]:
market["MntTotal"] = market["MntWines"] + market["MntFruits"] + market["MntMeatProducts"] + market["MntFishProducts"] + market["MntSweetProducts"] + market["MntGoldProds"]

**"MntTotal"** represent the amount spend in total (in all product)

_______

# Creation of the NumTotal column

In [66]:
market["NumTotalPurchase"] = market["NumDealsPurchases"] + market["NumWebPurchases"] + market["NumCatalogPurchases"] + market["NumStorePurchases"]

**"NumTotal "** represent the total number of purchases 

*We don't add the the number of visits of the website*

_______

# Creation of the Num_OldCmp_Accepted column

In [67]:
market["Num_OldCmp_Accepted"] = market["AcceptedCmp1"] + market["AcceptedCmp2"] + market["AcceptedCmp3"] + market["AcceptedCmp4"] + market["AcceptedCmp5"]

**"Num_OldCmp_Accepted"** represent the number of old campaigns the client had accepted.

_______

In [68]:
market.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,dilename,ChildsNumber,MntTotal,NumTotalPurchase,Num_OldCmp_Accepted
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,...,10,4,7,0,0,0,0,0,0,1,marketing_campaign_1.xlsx,0,1617,25,0
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,...,1,2,5,0,0,0,0,0,0,0,marketing_campaign_1.xlsx,2,27,6,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,...,2,10,4,0,0,0,0,0,0,0,marketing_campaign_1.xlsx,0,776,21,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,...,0,4,6,0,0,0,0,0,0,0,marketing_campaign_1.xlsx,1,53,8,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,...,3,6,5,0,0,0,0,0,0,0,marketing_campaign_1.xlsx,1,422,19,0


# To make an analysis of the amount spend, we transform this feature into a categorial.

# To do that we use a boolean indexing.

In [69]:
market.MntTotal.describe()

count    2214.000000
mean      606.595754
std       602.101211
min         5.000000
25%        69.000000
50%       396.500000
75%      1047.750000
max      2525.000000
Name: MntTotal, dtype: float64

We have the repartition of the value inside MntTotal, we will create our category with these informations :

We decide to have 4 categories :
> 0 -> 69   ;   69 -> 397   ;   397 -> 1048   ;   1048 -> 2525

In [70]:
market["MntTotal_bool"] = market["MntTotal"][:]

In [71]:
market.loc[market["MntTotal_bool"] <= 69, "MntTotal_bool"] = 1
market.loc[(market["MntTotal_bool"] > 69) & (market["MntTotal_bool"] <= 397), "MntTotal_bool"] = 2
market.loc[(market["MntTotal_bool"] > 397) & (market["MntTotal_bool"] <= 1048), "MntTotal_bool"] = 3
market.loc[market["MntTotal_bool"] > 1048, "MntTotal_bool"] = 4

In [72]:
market.MntTotal_bool.value_counts()

1    561
3    552
4    552
2    549
Name: MntTotal_bool, dtype: int64

In [73]:
market.head(15)

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,...,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,dilename,ChildsNumber,MntTotal,NumTotalPurchase,Num_OldCmp_Accepted,MntTotal_bool
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,...,4,7,0,0,0,0,0,0,1,marketing_campaign_1.xlsx,0,1617,25,0,4
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,...,2,5,0,0,0,0,0,0,0,marketing_campaign_1.xlsx,2,27,6,0,1
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,...,10,4,0,0,0,0,0,0,0,marketing_campaign_1.xlsx,0,776,21,0,3
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,...,4,6,0,0,0,0,0,0,0,marketing_campaign_1.xlsx,1,53,8,0,1
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,...,6,5,0,0,0,0,0,0,0,marketing_campaign_1.xlsx,1,422,19,0,3
5,7446,1967,Master,Together,62513.0,0,1,2013-09-09,16,520,42,98,0,42,14,...,10,6,0,0,0,0,0,0,0,marketing_campaign_1.xlsx,1,716,22,0,3
6,965,1971,Graduation,Divorced,55635.0,0,1,2012-11-13,34,235,65,164,50,49,27,...,7,6,0,0,0,0,0,0,0,marketing_campaign_1.xlsx,1,590,21,0,3
7,6177,1985,PhD,Married,33454.0,1,0,2013-05-08,32,76,10,56,3,1,23,...,4,8,0,0,0,0,0,0,0,marketing_campaign_1.xlsx,1,169,10,0,2
8,4855,1974,PhD,Together,30351.0,1,0,2013-06-06,19,14,0,24,3,3,2,...,2,9,0,0,0,0,0,0,1,marketing_campaign_1.xlsx,1,46,6,0,1
9,5899,1950,PhD,Together,5648.0,1,1,2014-03-13,68,28,0,6,1,1,13,...,0,20,1,0,0,0,0,0,0,marketing_campaign_1.xlsx,2,49,2,1,1
